In [88]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Lasso, Ridge



In [2]:

# Load the training and test data
df_train_orig = pd.read_csv('train.csv')
df_test_orig = pd.read_csv('test.csv')


In [26]:

# Make a copy of the original data
df_train = df_train_orig.copy()
df_test = df_test_orig.copy()
target_col = 'price_doc'
row_id_col = 'row ID'


In [31]:

# # Label encode the 'sub_area' column
# label_encoder = LabelEncoder()
# df_train['sub_area'] = label_encoder.fit_transform(df_train['sub_area'])
# df_test['sub_area'] = label_encoder.transform(df_test['sub_area'])


# RUN ABOVE OR BELOW CODE


# Calculate the counts of each category in the "sub_area" column
category_counts = df_train['sub_area'].value_counts()

# Identify the top 10 categories
top_categories = category_counts.head(100).index

# Replace categories not in the top 10 with "Other"
df_train.loc[~df_train['sub_area'].isin(top_categories), 'sub_area'] = 'Other'
df_test.loc[~df_test['sub_area'].isin(top_categories), 'sub_area'] = 'Other'

# Display the modified DataFrame
print(df_train.shape )
category_counts = df_train['sub_area'].value_counts()

# Display the counts of each category
category_counts_after_replacement = df_train['sub_area'].value_counts()
print(category_counts_after_replacement)

# after this dub area has 100 categories and rest are replaced by other


(181507, 272)
sub_area
Other                       75731
Poselenie Shhapovskoe        1105
Alekseevskoe                 1104
Tverskoe                     1101
Poselenie Mosrentgen         1099
                            ...  
Poselenie Filimonkovskoe     1051
Pechatniki                   1050
Marfino                      1050
Orehovo-Borisovo Juzhnoe     1049
Chertanovo Juzhnoe           1049
Name: count, Length: 100, dtype: int64


In [32]:
print(df_train.shape)
print(df_test.shape)

(181507, 272)
(77789, 272)


In [87]:

# Create dummy variables for categorical features
X_train_full = pd.get_dummies(df_train.drop(columns=[target_col]), drop_first=True)
y = df_train[target_col]

X_test = pd.get_dummies(df_test, drop_first=True)

# Extract the 'row ID' column for later use and remove it from X_test
row_ids = df_test['row ID']
X_test.drop(['row ID'], axis=1, inplace=True)

print(X_train_full.shape)
print(X_test.shape)

(181507, 372)
(77789, 372)


In [ ]:
model = DecisionTreeRegressor(max_depth=10, random_state=42)
# Fit the model to the training data
model.fit(X_train_full, y)

# Get feature importances
feature_importances = model.feature_importances_

# Sort features by importance
sorted_indices = np.argsort(feature_importances)[::-1]
print(sorted_indices)


In [90]:
# Assuming X_train_full is a pandas DataFrame
top_100_feature_names = X_train_full.columns[sorted_indices[:100]]
print(top_100_feature_names)


Index(['mosque_count_500', 'culture_objects_top_25_raion', 'full_sq',
       'cafe_count_3000_price_2500', 'sport_count_3000', 'cafe_count_3000',
       'culture_objects_top_25_yes', 'cafe_count_5000_price_high',
       'cafe_count_2000_price_4000', 'indust_part', 'sport_count_2000',
       'university_km', 'green_part_1500', 'life_sq',
       'raion_build_count_with_builddate_info', 'sport_count_500',
       'build_count_1946-1970', 'church_count_1500',
       'cafe_count_3000_price_high', 'office_sqm_5000', 'female_f',
       'sadovoe_km', 'cafe_count_1000_price_1500', 'catering_km',
       'cemetery_km', 'cafe_count_3000_na_price', 'church_count_3000',
       'office_count_1500', 'trc_count_1500', 'market_count_1000',
       'cafe_count_500_price_2500', 'ID_railroad_station_avto',
       'cafe_count_1500_price_high', 'green_part_2000', 'zd_vokzaly_avto_km',
       'prom_part_3000', 'exhibition_km', 'cafe_count_5000_price_1000',
       'museum_km', 'cafe_count_500_price_1000', 'cafe_

In [91]:
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y, test_size=0.05, random_state=42)


In [92]:
X_train_top100 = X_train[top_100_feature_names]
X_val_top100 = X_val[top_100_feature_names]
X_test_top100 = X_test[top_100_feature_names]

print(X_train_top100.shape)
print(X_val_top100.shape)
print(X_test_top100.shape)

(172431, 100)
(9076, 100)
(77789, 100)


In [93]:
selector = SequentialFeatureSelector(estimator=LinearRegression(), n_features_to_select=20)
selector.fit(X_train_top100, y_train)


SequentialFeatureSelector(estimator=LinearRegression(), n_features_to_select=20)

In [94]:

# Get the selected feature indices
selected_feature_indices = selector.get_support()

# Select the 20 best features
X_train_top20 = X_train_top100.iloc[:, selected_feature_indices]
X_val_top20 = X_val_top100.iloc[:, selected_feature_indices]
X_test_top20 = X_test_top100.iloc[:, selected_feature_indices]

print(X_train_top20.shape)
print(X_val_top20.shape)
print(X_test_top20.shape)

(172431, 20)
(9076, 20)
(77789, 20)


In [100]:
poly = PolynomialFeatures(degree=2, interaction_only=True)
X_train_poly = poly.fit_transform(X_train_top20)
X_val_poly = poly.transform(X_val_top20)
X_test_poly = poly.transform(X_test_top20)

print(X_train_poly.shape)
print(X_val_poly.shape)
print(X_test_poly.shape)

(172431, 211)
(9076, 211)
(77789, 211)


In [101]:
pca = PCA(n_components=200)
X_train_pca = pca.fit_transform(X_train_poly)
X_val_pca = pca.transform(X_val_poly)
X_test_pca = pca.transform(X_test_poly)


In [102]:
print(X_train_pca.shape)
print(X_val_pca.shape)
print(X_test_pca.shape)


(172431, 200)
(9076, 200)
(77789, 200)


In [103]:


model = Ridge()
model.fit(X_train_pca, y_train)

# Evaluate the model on the validation set
predictions = model.predict(X_val_pca)
rmse = np.sqrt(mean_squared_error(y_val, predictions))
print(f"RMSE on validation set:", rmse)


RMSE on validation set: 13395237.808008611


In [104]:
test_predictions = model.predict(X_test_pca)

# Create a DataFrame with 'row ID' and predictions
result_df = pd.DataFrame({'row ID': row_ids, 'price_doc': test_predictions})

# Save the results to a CSV file
result_df.to_csv('predictions_23.csv', index=False)

In [77]:
import pandas as pd
from sklearn.feature_selection import VarianceThreshold

threshold = 1

selector = VarianceThreshold(threshold=threshold)
X_filtered = selector.fit_transform(X_train_full)

selected_feature_indices = selector.get_support()
selected_feature_names = X_train_full.columns[selected_feature_indices]

X_train_full = X_train_full[selected_feature_names]
X_test = X_test[selected_feature_names]

print(X_train_full.shape)
print(X_test.shape)

(181507, 242)
(77789, 242)


In [82]:

# Scale the values
scaler = StandardScaler()
X_train_full = scaler.fit_transform(X_train_full)
X_test = scaler.transform(X_test)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y, test_size=0.05, random_state=42)


In [ ]:

selector = SequentialFeatureSelector(estimator=LinearRegression(), n_features_to_select=10)

# Fit the selector to the training data
selector.fit(X_train, y_train)

# Get the selected feature indices
selected_feature_indices = selector.k_feature_idx_

In [52]:
pca = PCA(n_components=250)  # Adjust the number of components as needed
X_train_pca = pca.fit_transform(X_train)

# Apply PCA on the scaled validation data
X_val_pca = pca.transform(X_val)

X_test_pca = pca.transform(X_test)

In [20]:
model = LinearRegression()

num_features_to_select = 30  # You can adjust the number of features you want to select
rfe = RFE(model, n_features_to_select=num_features_to_select, step=2)

# Fit the RFE to your data
rfe.fit(X_train_pca, y_train)

# Get the selected feature indices
selected_feature_indices = np.where(rfe.support_)[0]

In [23]:
X_train_pca = X_train_pca[:, selected_feature_indices]
X_val_pca = X_val_pca[:, selected_feature_indices]
X_test_pca = X_test_pca[:, selected_feature_indices]

print(X_train_pca.shape)
print(X_val_pca.shape)
print(X_test_pca.shape)

(172431, 30)
(9076, 30)
(77789, 30)


In [53]:

# Train the linear regression model
model = LinearRegression()
model.fit(X_train_pca, y_train)

# Evaluate the model on the validation set
predictions = model.predict(X_val_pca)
rmse = np.sqrt(mean_squared_error(y_val, predictions))
print(f"RMSE on validation set:", rmse)

# Make predictions on the test data
# test_predictions = model.predict(X_test)

# # Create a DataFrame with 'row ID' and predictions
# result_df = pd.DataFrame({'row ID': row_ids, 'price_doc': test_predictions})

# # Save the results to a CSV file
# result_df.to_csv('predictions_linear_regression.csv', index=False)


RMSE on validation set: 13374208.247026734


In [54]:

# Make predictions on the test data
test_predictions = model.predict(X_test_pca)

# Create a DataFrame with 'row ID' and predictions
result_df = pd.DataFrame({'row ID': row_ids, 'price_doc': test_predictions})

# Save the results to a CSV file
result_df.to_csv('predictions_22.csv', index=False)